<a href="https://colab.research.google.com/github/hmin27/2023_DL_Clip/blob/main/fewshot_dataset.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [7]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [10]:
import os
import shutil
import json
import random

full_dataset_path = '/content/drive/MyDrive/Study/DL_CLIP/Food'
few_shot_dataset_path = '/content/drive/MyDrive/Study/DL_CLIP/Food_fewshot'

classes = os.listdir(full_dataset_path)

os.makedirs(few_shot_dataset_path, exist_ok=True)
num_shots = [1, 2, 4, 8, 16]

for n_shots in num_shots:
    # Create subdirectories for each shot level
    n_shot_subdir = f"{n_shots}shot"
    n_shot_dir = os.path.join(few_shot_dataset_path, n_shot_subdir)
    os.makedirs(n_shot_dir, exist_ok=True)

    for class_name in classes:
        class_path = os.path.join(full_dataset_path, class_name)
        few_shot_class_path = os.path.join(n_shot_dir, class_name)

        os.makedirs(few_shot_class_path, exist_ok=True)

        # Get all available samples in the class
        all_samples = os.listdir(class_path)
        selected_samples = random.sample(all_samples, n_shots)

        for sample in selected_samples:
            src_path = os.path.join(class_path, sample)
            dst_path = os.path.join(few_shot_class_path, sample)
            shutil.copy(src_path, dst_path)

few_shot_info = {
    "num_shots": num_shots,
    "classes": classes
}

with open(os.path.join(few_shot_dataset_path, "few_shot_info.json"), "w") as f:
    json.dump(few_shot_info, f)